In [1]:
{-# LANGUAGE OverloadedStrings #-}

import qualified Data.ByteString                  as B
import qualified Data.ByteString.Char8            as B (lines)
import qualified Data.Attoparsec.ByteString       as A
import qualified Data.Attoparsec.ByteString.Char8 as C

input <- B.lines <$> B.readFile "../input/day21.txt"

In [2]:
data Instr
    = SwpP Int  Int
    | SwpL Char Char
    | RotL Int
    | RotR Int
    | RotB Char
    | RevP Int Int
    | Move Int Int
    deriving (Eq, Show)


swpP = do
    A.string "swap position "
    start <- C.decimal
    A.string " with position "
    end <- C.decimal
    return $ SwpP start end

swpL = do
    A.string "swap letter "
    start <- C.anyChar
    A.string " with letter "
    end  <- C.anyChar
    return $ SwpL start end

rotL = do
    A.string "rotate left "
    steps <- C.decimal
    A.choice [A.string " steps", A.string " step"]
    return $ RotL steps

rotR = do
    A.string "rotate right "
    steps <- C.decimal
    A.choice [A.string " steps", A.string " step"]
    return $ RotR steps

rotB = do
    A.string "rotate based on position of letter "
    letter <- C.anyChar
    return $ RotB letter

revP = do
    A.string "reverse positions "
    x <- C.decimal
    A.string " through "
    y <- C.decimal
    return $ RevP x y

move = do
    A.string "move position "
    x <- C.decimal
    A.string " to position "
    y <- C.decimal
    return $ Move x y

parser = A.choice [swpP, swpL, rotL, rotR, rotB, revP, move]

parsed = map (either error id . A.parseOnly parser) input

test = map (either error id . A.parseOnly parser)
    [ "swap position 4 with position 0"
    , "swap letter d with letter b"
    , "reverse positions 0 through 4"
    , "rotate left 1 step"
    , "move position 1 to position 4"
    , "move position 3 to position 0"
    , "rotate based on position of letter b"
    , "rotate based on position of letter d"
    ]

In [44]:
import qualified Data.Vector.Unboxed as V
import Data.List (foldl', inits)

interpret vector (SwpP x y) = let
    xC = vector V.! x
    yC = vector V.! y
    in vector V.// [(x,yC), (y,xC)]

interpret vector (SwpL a b) = let
    Just x = V.elemIndex a vector
    Just y = V.elemIndex b vector
    in vector V.// [(x, b), (y, a)]

interpret vector (RotL r) = let
    (t, h) = V.splitAt r vector
    in h V.++ t

interpret vector (RotR r) = let
    mx = V.length vector
    r' = r `mod` mx
    (t,h) = V.splitAt (mx-r') vector
    in h V.++ t

interpret vector (RotB c) = let
    Just i = V.elemIndex c vector
    i' = if i>=4 then i+2 else i+1
    in interpret vector (RotR i')

interpret vector (RevP x y) = let
    y'    = (y-x)+1
    slice = V.reverse $ V.slice x y' vector
    h     = V.take x vector
    t     = V.drop (y+1) vector
    in h V.++ slice V.++ t

interpret vector (Move x y) = let
    c       = vector V.! x
    (h, t)  = V.splitAt x vector
    new     = h V.++ V.tail t
    (h',t') = V.splitAt y new
    in h' V.++ V.cons c t'

map (foldl' interpret (V.fromList "abcde")) (inits test)

["abcde","ebcda","edcba","abcde","bcdea","bdeac","abdec","ecabd","decab"]

In [48]:
foldl' interpret (V.fromList "abcdefgh") parsed

"hcdefbag"

In [51]:
reverseInstr vector instr = case instr of
    SwpP x y -> interpret vector $ SwpP x y
    SwpL a b -> interpret vector $ SwpL a b
    RotL l   -> interpret vector $ RotR l
    RotR r   -> interpret vector $ RotL r
    RevP x y -> interpret vector $ RevP x y
    Move x y -> interpret vector $ Move y x
    RotB c   -> loop (interpret vector (RotL 1)) c
    where loop vector' c = if interpret vector' (RotB c) == vector
            then vector'
            else loop (interpret vector' (RotL 1)) c

foldl' reverseInstr (V.fromList "fbgdceah") (reverse parsed)

"fbhaegdc"